# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [ ]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [ ]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [ ]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'brand page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [ ]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-Image
Updated
3 days ago
•
6.64k
•
822
Lightricks/LTX-2
Updated
about 4 hours ago
•
1.54M
•
1.14k
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
11 days ago
•
53.1k
•
747
openbmb/AgentCPM-Explore
Updated
about 7 hours ago
•
1.55k
•
337
google/translategemma-4b-it
Updated
3 days ago
•
12.8k
•
304
Browse 2M+ models
Spaces
Running
on
Zero
Featured
914
Qwen Image Multiple Angles 3D Camera
🎥
914
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.58k
Z Image Turbo
🖼
1.58k
Generate stunning 

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-Image\nUpdated\n3 days ago\n•\n6.64k\n•\n822\nLightricks/LTX-2\nUpdated\nabout 4 hours ago\n•\n1.54M\n•\n1.14k\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n11 days ago\n•\n53.1k\n•\n747\nopenbmb/AgentCPM-Explore\nUpdated\nabout 7 hours ago\n•\n1.55k\n•\n337\ngoogle/translategemma-4b-it\nUpdated\n3 days ago\n•\n12.8k\n•\n304\nBrowse 2M+ models\nSpaces\nRunning\non\nZe

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning. It serves as a vibrant collaboration platform where machine learning engineers, scientists, and enthusiasts worldwide come together to share, explore, and experiment with models, datasets, and applications. The platform accelerates innovation and fosters transparency, openness, and ethical AI development.

---

## What We Offer

- **Models:** Access and browse over 2 million machine learning models spanning various modalities including text, image, video, audio, and even 3D. Stay on the cutting edge with trending models updated frequently by the community.

- **Datasets:** Explore a rich repository of over 500,000 datasets curated from top contributors to power diverse machine learning tasks.

- **Spaces:** Interactive AI applications and demos powered by community projects, offering hands-on experiences such as 3D image manipulation, text-to-image generation, and more.

- **Community:** An inclusive and growing global platform for developers and researchers to collaborate, build portfolios, and share their advancements with the world.

- **Enterprise Solutions:** Scalable paid compute and enterprise-grade solutions to help organizations accelerate their machine learning projects seamlessly.

---

## Company Culture & Philosophy

Hugging Face believes in an open, collaborative, and ethical AI future. It empowers the next generation of machine learning professionals by providing:

- A **central hub** for hosting, collaboration, and discovery of open-source machine learning work.
- Support for **open source** development to drive faster innovation and knowledge sharing.
- A culture that values **transparency, inclusivity, and community engagement**.
- Resources and tools aimed at **building personal ML portfolios and advancing careers** in AI.

---

## Customers & Community

Hugging Face serves a diverse user base including:

- Individual ML engineers and researchers looking to contribute and explore cutting-edge models.
- Academic institutions accessing datasets and models for research and education.
- Enterprises seeking robust ML infrastructure with compute and deployment solutions.
- Developers building AI applications supported by the dynamic Hugging Face ecosystem.

Highly active community contributions and trending models demonstrate the platform’s role as a trusted hub for ML innovation.

---

## Careers at Hugging Face

Join a forward-thinking, global AI community dedicated to shaping the future of machine learning. Hugging Face offers opportunities to:

- Work on open-source initiatives impacting millions of users.
- Collaborate with top ML talent and thought leaders.
- Contribute to ethical and responsible AI technologies.
- Grow your career in a vibrant, innovative, and inclusive environment.

---

## Brand Identity

- Signature colors: Yellow (#FFD21E), Orange (#FF9D00), Grey (#6B7280)
- Logo available in .svg, .png, and .ai formats reflecting the brand’s vibrant and dynamic image.

---

## Get Involved

- **Sign up** on the Hugging Face platform to start exploring models, datasets, and AI apps.
- **Host and collaborate** on your machine learning projects.
- **Build your profile** and share your work with a global community.
- Access extensive **documentation, guides, and support** for all users.

---

For more information, visit: [huggingface.co](https://huggingface.co)  

Build the future of AI with Hugging Face — together.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the vibrant AI community and platform at the forefront of building the future of machine learning. It serves as a central hub where machine learning engineers, scientists, and enthusiasts collaborate on models, datasets, and applications to create an open and ethical AI ecosystem.

The company empowers the next generation of AI professionals with accessible tools, extensive open-source resources, and a thriving community that encourages sharing, learning, and innovation. Hugging Face stands at the core of the AI revolution, fostering collaboration that spans text, image, video, audio, and even 3D modalities.

---

## Platform & Offerings

- **Model Hub:** Browse and access over 2 million machine learning models across many domains.
- **Datasets:** Explore over 500,000 datasets to train and evaluate models.
- **Spaces:** Interactive ML applications created by the community, with features like zero-shot deployment using Zero GPU.
- **Community:** A fast-growing, engaged user base driving collaboration and knowledge sharing through forums, GitHub, Discord, and social channels.
- **Open Source Stack:** Provides tools and libraries for fast, scalable, and open machine learning development.
- **Enterprise Solutions:** Tailored plans for teams and organizations featuring:
  - Enterprise-grade security  
  - Single Sign-On (SSO) integration  
  - Granular access controls and audit logs  
  - Analytics dashboards  
  - Advanced compute scaling including ZeroGPU quota boosts  
  - Private storage and dataset viewers  
  - Comprehensive resource and token management

---

## Company Culture

Hugging Face promotes an open and ethical AI future built on collaboration, transparency, and inclusivity. The community-focused culture invites contributors worldwide to share models, datasets, and apps, forging a decentralized ecosystem of AI innovation. Their team of talented scientists and engineers work at the cutting edge of technology while nurturing the growth of the AI community globally.

Values include:  
- Openness and transparency  
- Community empowerment  
- Ethical AI development  
- Innovation through collaboration

---

## Customers & Community

Hugging Face serves a diverse global audience including individual researchers, AI practitioners, startups, and large enterprises. The platform supports a range of use cases from academic research to commercial AI product development, offering free public resources as well as premium enterprise services.

Notable users include AI teams from top tech companies, research institutions, and production-scale businesses leveraging Hugging Face’s tools to accelerate innovation.

---

## Careers at Hugging Face

Join Hugging Face to work at the heart of the AI community revolution. The company values passionate, talented individuals who want to shape the future of AI in an open and collaborative environment.

Open roles typically include:  
- Machine Learning Engineers  
- Research Scientists  
- Software Engineers  
- Community Managers  
- Enterprise Solutions Specialists  

Hugging Face offers opportunities to grow professionally while contributing to cutting-edge AI projects and the broader machine learning ecosystem.

---

## Get Involved

- **Explore Models & Datasets:** Dive into millions of open resources to build and experiment.  
- **Join the Community:** Engage on forums, Discord, GitHub, and social media to collaborate and learn.  
- **Use Enterprise Plans:** Scale AI projects securely with enterprise-grade tools and support.  
- **Contribute & Share:** Upload your models, datasets, or Spaces apps to build your ML profile and showcase your work.

---

For more information and to join the AI community shaping tomorrow’s technology, visit [huggingface.co](https://huggingface.co).

---

**Hugging Face – The AI community building the future.**

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>